In [2]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib import cm

In [3]:
# Matrix preparation auxiliary functions

def prep_diags(N, d):
    # This performs the delta(x-x'+da) operation for the matrix
    if d == 0:
        return np.eye(N, dtype=complex)
    else:
        mat = np.zeros((N, N), dtype=complex)
        if d == 1:
            for i in range(N - 1):
                mat[i + 1][i] = 1
        else: # d == - 1
            for i in range(N - 1):
                mat[i][i + 1] = 1
        return mat

def mat2hext(mat, q):
    N = mat.shape[0]
    board = np.zeros((4 * N, 4 * N), dtype=complex)
    board[((q // 4) * N):((q // 4 + 1) * N), ((q % 4) * N):((q % 4 + 1) * N)] = mat
    return board

def dot_pauli(mat, pauli_idxs):
    board = np.zeros((4 * mat.shape[0], 4 * mat.shape[0]), dtype=complex)
    if pauli_idxs == '1, x':
        board += mat2hext(mat, 1) + mat2hext(mat, 4) + mat2hext(mat, 11) + mat2hext(mat, 14)
    elif pauli_idxs == '1, y':
        board += -1j * mat2hext(mat, 1) + 1j * mat2hext(mat, 4) + -1j * mat2hext(mat, 11) + 1j * mat2hext(mat, 14)
    elif pauli_idxs == '1, z':
        board += mat2hext(mat, 0) - mat2hext(mat, 5) + mat2hext(mat, 10) - mat2hext(mat, 15)
    elif pauli_idxs == 'x, 1':
        board += mat2hext(mat, 2) + mat2hext(mat, 7) + mat2hext(mat, 8) + mat2hext(mat, 13)
    elif pauli_idxs == 'z, 1':
        board += mat2hext(mat, 0) + mat2hext(mat, 5) - mat2hext(mat, 10) - mat2hext(mat, 15)
    elif pauli_idxs == 'z, x':
        board += mat2hext(mat, 1) + mat2hext(mat, 4) - mat2hext(mat, 11) - mat2hext(mat, 14)
    elif pauli_idxs == 'z, y':
        board += -1j * mat2hext(mat, 1) + 1j * mat2hext(mat, 4) + 1j * mat2hext(mat, 11) - 1j * mat2hext(mat, 14)
    return np.matrix(board)

def fourier_ham(N, b, M, ky, t):
    return b * np.sin(ky) * dot_pauli(prep_diags(N, 0), 'z, x') + \
           b * (-1 / 2j) * dot_pauli(prep_diags(N, 1) - prep_diags(N, -1), 'z, y') + \
           dot_pauli((M - 2 * t * np.cos(ky)) * prep_diags(N, 0) - t * (prep_diags(N, 1) + prep_diags(N, -1)), 'x, 1')


In [12]:
# Predef

M = 2
N = 20
ks = np.linspace(-np.pi, np.pi, N, endpoint=False)

eigvls = np.zeros((len(ks), 4 * N), dtype=complex)
eigvs = np.zeros((len(ks), 4 * N, 4 * N), dtype=complex)

for iy, ky in enumerate(ks):
    hamiltonian = fourier_ham(N, 1, M, ky, 1)
    # According to documentation, the eigenvectors are sorted by eigenvalue, so the last one is the + band vector
    eigvls[iy, :], cur_eigvs = np.linalg.eigh(hamiltonian)
    eigvs[iy, :, :] = cur_eigvs

for band in range(4 * N):
    if band == 2 * N - 1 or band == 2 * N - 2: # Just verifying they are degenerate
        print(eigvls[:, band])
    plt.plot(ks, eigvls[:, band])
plt.title(f"Band Structure for M=2t")
plt.ylabel("Energy [arb. units]")
plt.xlabel("ky")
plt.show()

[-2.02702410e+00+0.j -1.95395164e+00+0.j -1.74787093e+00+0.j
 -1.45182542e+00+0.j -1.15396446e+00+0.j -1.00333581e+00+0.j
 -9.51056516e-01+0.j -8.09016994e-01+0.j -5.87785252e-01+0.j
 -3.09016994e-01+0.j -2.25801171e-05+0.j -3.09016994e-01+0.j
 -5.87785252e-01+0.j -8.09016994e-01+0.j -9.51056516e-01+0.j
 -1.00333581e+00+0.j -1.15396446e+00+0.j -1.45182542e+00+0.j
 -1.74787093e+00+0.j -1.95395164e+00+0.j]
[-2.02702410e+00+0.j -1.95395164e+00+0.j -1.74787093e+00+0.j
 -1.45182542e+00+0.j -1.15396446e+00+0.j -1.00333581e+00+0.j
 -9.51056516e-01+0.j -8.09016994e-01+0.j -5.87785252e-01+0.j
 -3.09016994e-01+0.j -2.25801171e-05+0.j -3.09016994e-01+0.j
 -5.87785252e-01+0.j -8.09016994e-01+0.j -9.51056516e-01+0.j
 -1.00333581e+00+0.j -1.15396446e+00+0.j -1.45182542e+00+0.j
 -1.74787093e+00+0.j -1.95395164e+00+0.j]


In [14]:
eigvs_rot = np.zeros((len(ks), 4 * N, 2), dtype=complex)

def expec(vec, mat):
    return np.real(vec.conj().T @ mat @ vec)

for iy, ky in enumerate(ks):
    vedge1 = eigvs[iy, :, 2 * N - 2]
    vedge2 = eigvs[iy, :, 2 * N - 1]
    subspace = np.column_stack([vedge1, vedge2])
    Sx_proj = subspace.conj().T @ dot_pauli(prep_diags(N, 0), '1, x') @ subspace

    w, U = np.linalg.eigh(Sx_proj)
    vedge1_rot, vedge2_rot = subspace @ U[:, 0], subspace @ U[:, 1]

    print(f"For ky={ky}:")
    for state in [vedge1_rot, vedge2_rot]:
        Sx_exp = expec(state, dot_pauli(prep_diags(N, 0), '1, x'))
        Sy_exp = expec(state, dot_pauli(prep_diags(N, 0), '1, y'))
        Sz_exp = expec(state, dot_pauli(prep_diags(N, 0), '1, z'))
        print(Sx_exp, Sy_exp, Sz_exp)

    if expec(vedge2_rot, dot_pauli(prep_diags(N, 0), '1, x')) < 0:
        temp = vedge1_rot
        vedge1_rot = vedge2_rot
        vedge2_rot = temp

    # Now for clarity, rotate the eigenvectors to the s_x basis
    R = 1 / np.sqrt(2) * (dot_pauli(prep_diags(N, 0), '1, x') + dot_pauli(prep_diags(N, 0), '1, z'))
    vedge1_xbasis = R @ vedge1_rot
    vedge2_xbasis = R @ vedge2_rot
    subspace_rot = np.column_stack([vedge1_xbasis, vedge2_xbasis])
    eigvs_rot[iy, :, :] = subspace_rot

For ky=-3.141592653589793:
[[-0.9978133]] [[0.]] [[-3.82506526e-16]]
[[0.9978133]] [[0.]] [[-1.78676518e-16]]
For ky=-2.827433388230814:
[[-0.99759894]] [[0.]] [[-1.07552856e-16]]
[[0.99759894]] [[0.]] [[5.6378513e-17]]
For ky=-2.5132741228718345:
[[-0.99676185]] [[0.]] [[-1.98625838e-16]]
[[0.99676185]] [[0.]] [[4.07660017e-16]]
For ky=-2.199114857512855:
[[-0.99422707]] [[0.]] [[-6.80878964e-16]]
[[0.99422707]] [[0.]] [[4.88324658e-16]]
For ky=-1.8849555921538759:
[[-0.98279633]] [[0.]] [[-2.7451999e-16]]
[[0.98279633]] [[0.]] [[3.48245738e-16]]
For ky=-1.5707963267948966:
[[-0.71303685]] [[0.]] [[-3.09127723e-15]]
[[0.71303685]] [[0.]] [[2.28636554e-15]]
For ky=-1.2566370614359172:
[[-0.00020653]] [[0.]] [[8.32667268e-17]]
[[0.00020653]] [[0.]] [[-3.60822483e-16]]
For ky=-0.9424777960769379:
[[-0.00024776]] [[0.]] [[-4.16333634e-16]]
[[0.00024776]] [[0.]] [[-2.49800181e-16]]
For ky=-0.6283185307179586:
[[-0.00027707]] [[0.]] [[-2.77555756e-17]]
[[0.00027707]] [[0.]] [[2.49800181e-16

In [15]:
X, Y = np.meshgrid(np.arange(0, N, 1), ks)

fig, axs = plt.subplots(2, 2, subplot_kw={"projection": "3d"})
fig.suptitle(f"Eigenfunctions for M={M}")

surf = axs[0, 0].plot_surface(X, Y, np.abs(eigvs_rot[:, :N, 0]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[0, 0].set_xlabel('x (site)')
axs[0, 0].set_ylabel('ky')
axs[0, 0].title.set_text('Sublattice A, spin x up')

surf = axs[0, 1].plot_surface(X, Y, np.abs(eigvs_rot[:, N:2*N, 0]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[0, 1].set_xlabel('x (site)')
axs[0, 1].set_ylabel('ky')
axs[0, 1].title.set_text('Sublattice A, spin x down')

surf = axs[1, 0].plot_surface(X, Y, np.abs(eigvs_rot[:, 2*N:3*N, 0]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[1, 0].set_xlabel('x (site)')
axs[1, 0].set_ylabel('ky')
axs[1, 0].title.set_text(f"Sublattice B, spin x up")

surf = axs[1, 1].plot_surface(X, Y, np.abs(eigvs_rot[:, 3*N:, 0]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[1, 1].set_xlabel('x (site)')
axs[1, 1].set_ylabel('ky')
axs[1, 1].title.set_text(f"Sublattice B, spin x down")

plt.show()

X, Y = np.meshgrid(np.arange(0, N, 1), ks)

fig, axs = plt.subplots(2, 2, subplot_kw={"projection": "3d"})
fig.suptitle(f"Eigenfunctions for M={M}")

surf = axs[0, 0].plot_surface(X, Y, np.abs(eigvs_rot[:, :N, 1]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[0, 0].set_xlabel('x (site)')
axs[0, 0].set_ylabel('ky')
axs[0, 0].title.set_text('Sublattice A, spin x up')

surf = axs[0, 1].plot_surface(X, Y, np.abs(eigvs_rot[:, N:2*N, 1]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[0, 1].set_xlabel('x (site)')
axs[0, 1].set_ylabel('ky')
axs[0, 1].title.set_text('Sublattice A, spin x down')

surf = axs[1, 0].plot_surface(X, Y, np.abs(eigvs_rot[:, 2*N:3*N, 1]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[1, 0].set_xlabel('x (site)')
axs[1, 0].set_ylabel('ky')
axs[1, 0].title.set_text(f"Sublattice B, spin x up")

surf = axs[1, 1].plot_surface(X, Y, np.abs(eigvs_rot[:, 3*N:, 1]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
axs[1, 1].set_xlabel('x (site)')
axs[1, 1].set_ylabel('ky')
axs[1, 1].title.set_text(f"Sublattice B, spin x down")

plt.show()